In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

In [2]:
def dfFromURL(url, tableNumber=1):
        # Parse the HTML as a string
        soup = BeautifulSoup(requests.get(url).content, 'lxml')
        tables = soup.find_all('table')
        # check table number is within number of tables on the page
        assert len(tables) >= tableNumber
        df = pd.read_html(str(tables[tableNumber-1]))[0]
       
        # Dropping the duplicate header for each table
        df.drop_duplicates(keep=False, inplace=True)
        # Removing the '.'  from date (example( 08. January 2021 will convert to 08 january 2021))
        df = df[:1]
        df['date'] = df['date'].apply(lambda x: x[0:2]+x[3:])
        # Removing the zero(0) from the start of date
        df['date'] = np.where((df['date'].str.startswith('0')),
                            df['date'].apply(lambda x: x[1:]), df['date'])
        # changing the stings to proper datetime format
        df['date'] = pd.to_datetime(df['date'], format="%d %B %Y")
        df.rename(columns=({'LME Copper Cash-Settlement': 'csp',
                'LME Copper 3-month': 'fut', 'LME Copper stock': 'stock'}), inplace=True)
        df.sort_values(by='date', inplace=True)
        df.reset_index(inplace=True)
        df.drop(columns='index', inplace=True)
        return df 

In [3]:
url='https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Cu_cash'
df=dfFromURL(url)

In [4]:
df

,date,csp,fut,stock
0,2022-06-01,9455.00,9455.00,145950


In [11]:
df['csp']

0    9455.00
Name: csp, dtype: object